In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

## Loading Environment File

In [ ]:
secret_path = Path("../.env")
print("Looking for dev.env at:", secret_path.resolve())
if secret_path.exists():
    load_dotenv(secret_path)
    print("Loaded environment variables from dev.env")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment variables or dev.env file.")
print("GOOGLE_API_KEY loaded:", bool(GOOGLE_API_KEY))

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables or dev.env file.")
print("OPENROUTER_API_KEY loaded:", bool(OPENROUTER_API_KEY))

## Setup: Imports, Retry Config

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.function_tool import FunctionTool
from google.genai import types
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext

import sqlite3

print("✅ ADK components imported successfully.")

Add the project root to sys.path to easily import our classes:

In [ ]:
import sys

# Ensure the project root (the parent of the "vaxtalk" directory) is on sys.path
# so that "import vaxtalk.model" finds the vaxtalk package under the project root.
project_root = Path.cwd().parent
vaxtalk_dir = project_root / "vaxtalk"

project_root_path = str(project_root.resolve())
if project_root_path not in sys.path:
    sys.path.insert(0, project_root_path)

from vaxtalk.model import Intensity, SentimentOutput
from vaxtalk.model.rag_output import RagOutput

Create a general retry policy:

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agents Creation

In [ ]:
# RAG system
rag_agent = Agent(
    name="RAG_Vaccine_Informer",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    instruction="""XXX""",
    tools=[],
    output_key="rag_output",  # The result of this agent will be stored in the session state with this key.
    output_schema=RagOutput,  # Define the expected output schema
)

print("✅ RAG created.")

In [ ]:
# Test: we could use a tool to save sentiment in session state.
@FunctionTool
def save_sentiment(
    tool_context: ToolContext, sentiment: SentimentOutput
) -> dict[str, str]:
    """
    Tool to record and save the sentiment analysis result into session state.

    Args:
        sentiment (SentimentOutput): The sentiment analysis result to be saved.
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["sentiment"] = sentiment

    return {"status": "success"}

print("✅ Sentiment Tools created.")

In [ ]:
sentiment_agent = Agent(
    name="sentiment_analysis",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""XXX""",
    tools=[save_sentiment],
    output_key="sentiment_output",  # The result will be stored with this key.
    output_schema=SentimentOutput,  # Define the expected output schema.
)

print("✅ sentiment_agent created.")

In [ ]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these findings into a single executive summary:

    **RAG output:**
    {rag_output}
    
    **User sentiment analysis:**
    {sentiment_output}
""",
    output_key="final_output",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

In [ ]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_rag_sentiment_agent = ParallelAgent(
    name="ParallelRAGAndSentimentTeam",
    sub_agents=[sentiment_agent, rag_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_rag_sentiment_agent, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

## Engine Def

In [ ]:
APP_NAME = "VaxTalkAssistant"
SQL_ASYNC_DRIVER = "aiosqlite"
DB_NAME = "vaxtalk_sessions.db"
DB_URL = f"sqlite+{SQL_ASYNC_DRIVER}:///{DB_NAME}"  # Local SQLite file

Session Management:

In [ ]:
# InMemorySessionService stores conversations in RAM (temporary)
#session_service = InMemorySessionService()

# Persistent memory using a SQLite database
# SQLite database will be created automatically
session_service = DatabaseSessionService(db_url=DB_URL)

We configure an events compaction process to run silently in the background. The compaction process **doesn't delete old events; it replaces them with a single, new `Event` that contains the summary.** 

In [ ]:
events_compaction_config = EventsCompactionConfig(
    compaction_interval=3,  # Trigger compaction every 3 invocations
    overlap_size=1,  # Keep 1 previous turn for context
)

Application:

In [ ]:
application = App(
    name=APP_NAME,
    root_agent=root_agent,
    events_compaction_config=events_compaction_config
)

In [ ]:
runner = Runner(
    app=application, 
    session_service=session_service
)

## Testing

In [ ]:
response = await runner.run_debug(
    "Should I vaccinate?"
)

## Exploring Persisted State

Let's explore the state we saved on the database.
Notice that there are special **Compaction Events**, created with our compaction policy to summarize the context.

In [ ]:
def check_data_in_db():
    with sqlite3.connect(DB_NAME) as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

If we knew the session ID, we could explore the events directly from the session service:

In [ ]:
user_id = "user"
session_id = "debug_session_id"

# Get the final session state
final_session = await session_service.get_session(
    app_name=application.name,
    user_id=user_id,
    session_id=session_id,
)
if not final_session:
    raise ValueError("Final session not found.")

print("Searching for Compaction Summary Event")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("✅ SUCCESS! Found the Compaction Event:")
        print(f"Author: {event.author}")
        print(f"Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print("❌ No compaction event found.")